## Contents
- <a id='chh1' href='#iterator'>迭代器</a>
- [生成器](#generator)
    - [send()](#send)
    - [throw()](#throw)
    - [生成器应用](#apply)
        - [斐波那契数列](#fibonacci)

## <a id='iterator' href='#chh1'>迭代器</a>
迭代器与生成器的关系： 生成器属于迭代器     
迭代器本身是一个底层的特性和概念，在程序中可以不用它。但它为生成器这一更为有趣的特性提供了基础。   
Lazy懒惰特性:     
```
Central idea: a lazy factory
From the outside, the iterator is like a lazy factory that is idle until you ask it for a value, which is when it starts to buzz and produce a single value, after which it turns idle again.
```
<img src='./imgs/iterable_iter_generator.png'>
<i>from: <a target='blank' href="https://nvie.com/posts/iterators-vs-generators/">Iterables vs. Iterators vs. Generators</a></i>


In [2]:
# %load /Users/huangmegan/python/algorithm/temp/gen_class_countdown.py

'''
自定义倒计时迭代器
包含方法：
__next__
__iter__
'''


class CountDown(object):
	def __init__(self, n):
		self.step = n

	def __iter__(self):
		return self
	
	def __next__(self):
		if self.step <= 0 :
			raise StopIteration
		self.step -= 1
		return self.step


In [3]:
c = CountDown(10)

In [4]:
next(c)

9

In [5]:
[i for i in c]

[8, 7, 6, 5, 4, 3, 2, 1, 0]

## <a id='generator'>生成器</a>

### <a id='send'>send()方法</a>

In [6]:
def prepare(generator):
    "修饰生成器函数，返回一个已经启动的生成器"
    def wrapper(*args, **kwargs):
        gen = generator(*args, **kwargs)
        print('wrapper:',type(gen))
        next(gen)
        return gen
    return wrapper

@prepare
def generator():
    while True:
        x = yield
        if x:
            print('received:',x)
        else:
            print('received nothing')

In [7]:
g = generator()

wrapper: <class 'generator'>


In [8]:
type(g)

generator

In [9]:
g.send(120)

received: 120


### <a id='throw'>throw()方法</a>

In [10]:
@prepare
def gen():
    count = 0
    while True:  
        count += 1
        print('counting...') 
         
        try:
            x = yield 'yes'
        except Exception:
            print('yielded:', count)
            
        print('received:', x)
        

In [11]:
g = gen()

wrapper: <class 'generator'>
counting...


In [12]:
g.send(10)   #紧接着上一个被执行的yield之后，并执行下一个yield语句

received: 10
counting...


'yes'

In [13]:
g.throw(Exception)  #紧接着上一次执行的yield之后，并执行下一个yield

yielded: 2
received: 10
counting...


'yes'

In [14]:
next(g)       #紧接着上一个被执行的yield之后，并执行下一个yield语句

received: None
counting...


'yes'

## <a id='apply'>生成器应用</a>

### <a id='fibonacci'>斐波那契数列</a>

通过装饰器限制产出的斐波那契数列个数

弊端：无法灵活地用使用者控制输出序列个数

In [15]:
def count(n):
    def decorator(gen):
        def wrapper():
            g = gen()
            for i in range(n):
                print('counting:',i+1)
                yield next(g)
        return wrapper
    return decorator

@count(5)  
def fibonacci():
    a,b = 0,1
    while True:
        yield a
        a, b = b, a+b
        

In [16]:
list(fibonacci())  #list实际接收的只有yield语句的返回值

counting: 1
counting: 2
counting: 3
counting: 4
counting: 5


[0, 1, 1, 2, 3]

In [17]:
f = fibonacci()
for i in range(10):   #count()装饰器对原生成器进行了限制，原生成器是无限的，限制后的生成器只有5个
    print(next(f))    

counting: 1
0
counting: 2
1
counting: 3
1
counting: 4
2
counting: 5
3


StopIteration: 

#### 改进一：
改造生成器本身，限制序列个数

弊端：破坏了生成器的定义

In [18]:
def fibonacci_with_limit(n):
    a, b = 0, 1
    for i in range(n):
        yield a
        a, b = b, a+b

In [19]:
list(fibonacci_with_limit(10))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]

#### 改进二：
在生成器外面控制序列个数

In [20]:
def fibonacci():
    a, b = 0, 1
    while True:
        yield a
        a, b = b, a+b

def limit(generator, n):
    for i in range(n):
        yield next(generator)

In [21]:
list(limit(fibonacci(), 10))

[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]